# CLASIFICADOR DE TEXTO MEDIANTE ENFOQUE LEARNING WITH DISAGREEMENT TASK 5

## Importar Dependencias y Librerias

In [ ]:
# Instalacion de dependencias
!pip install pytorch-lightning
!pip install --upgrade accelerate
!pip install framework-reproducibility
!pip install transformers datasets
!pip install --upgrade numpy
!pip install --upgrade pandas
!pip install --upgrade scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 812.2/812.2 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 868.8/868.8 kB 8.7 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manyl

Reiniciar Entorno (Recomendable de hacer siempre despues de instalar dependencias)

In [ ]:
# Instalacion de librerias
import random
import torch
import numpy as np
import os
from pytorch_lightning import seed_everything
import matplotlib.pyplot as plt
import seaborn as sns
import re

seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)# Store the average loss after eachepoch so we can plot them.
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
os.environ["TF_DETERMINISTIC_OPS"] = "1" # See:https://github.com/NVIDIA/tensorflow-determinism#confirmed-current-gpu-specific-sources-of-non-determinism-with-solutions
seed_everything(42, workers=True)

from datasets import Dataset, DatasetDict, load_metric
import pandas as pd
import sklearn as sk
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, average_precision_score, f1_score
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModelForSequenceClassification, \
TrainingArguments, Trainer, pipeline, EarlyStoppingCallback

INFO:lightning_fabric.utilities.seed:Seed set to 42


In [ ]:
# Comprobacion GPU
# Check that pyTorch is identifying the GPU
if torch.cuda.device_count() > 0:
    # If a GPU is available, print its name
    print(f'GPU detected. Currently using: "{torch.cuda.get_device_name(0)}"')
    # Set the device to GPU for accelerated computations
    device = torch.device("cuda")
else:
    # If no GPU is available, inform the user to change the runtime type
    print('Currently using CPU. To utilize GPU acceleration, change the runtime type in the \'runtime\' tab.')

Currently using CPU. To utilize GPU acceleration, change the runtime type in the 'runtime' tab.


In [ ]:
# Conexion drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Lectura y Etiquetado de Datos

Cargar los datos de entrenamiento y test

In [ ]:
# En este caso, tenemos un único fichero de entrenamiento y un fichero independiente de test
train_data_path = '/content/drive/MyDrive/Dataset/Task 5/train_original.json'
train_data_path = '/content/drive/MyDrive/Dataset/Task 5/train_simple.json'
train_data_path = '/content/drive/MyDrive/Dataset/Task 5/train_back.json'
test_data_path = '/content/drive/MyDrive/Dataset/Task 5/test_task5_hard.json'
#############################################################################################

# Los transformamos en Dataframes
dataset_txt_full = pd.read_json(train_data_path, orient='index')
dataset_txt_full

Split de los datos

In [ ]:
# Dividir los datos entre train, val y test
df = dataset_txt_full.sample(n=int(len(dataset_txt_full)), random_state=42)

train_df, val_df = train_test_split(df, test_size=0.15, shuffle=True, stratify=None, random_state=42)

print("Ejemplos del conjunto completo de entrenamiento: ", len(dataset_txt_full))
print("Ejemplos usados para entrenar: ", len(train_df))
print("Ejemplos usados para validar: ", len(val_df))

In [ ]:
val_df.reset_index(drop=True, inplace=True)
val_df

Hard voting val_df

In [ ]:
# Cuenta la etiqueta más usada
def most_common_label(labels):
    counts = {}
    for label in labels:
        counts[label] = counts.get(label, 0) + 1
    return max(counts, key=counts.get)

columna_labels_task5 = val_df['labels_task5']
mv = []
indices = []
i = 0

for columna in columna_labels_task5:
    labels = []
    for data in columna:
        if data == 'DIRECT':
            labels.append('DIRECT')
        elif data == 'JUDGEMENTAL':
            labels.append('JUDGEMENTAL')
    if labels != []:
        mas_usado = most_common_label(labels)
        mv.append(mas_usado)
        indices.append(i)
    i += 1

val_df = val_df.loc[indices]
val_df['labels_task5'] = mv
val_df

## Preprocesado de Datos

Funciones de limpieza

In [ ]:
import re

def remove_links(tweet):
    """Takes a string and removes web links from it"""
    tweet = re.sub(r'http\S+', '', tweet)        # remove http links
    tweet = re.sub(r'bit.ly/\S+', '', tweet)     # remove bitly links
    tweet = re.sub(r'\[link\]', '', tweet )      # remove [link]
    tweet = re.sub(r'\[url\]', '', tweet )       # remove [url]
    tweet = re.sub(r'pic.twitter\S+','', tweet)
    return tweet

def remove_users(tweet):
    """Takes a string and removes retweet and @user information"""
    tweet = re.sub('(RT\s@[A-Za-z]+[A-Za-z0-9-_]+)', '', tweet)  # remove re-tweet
    tweet = re.sub('(@[A-Za-z]+[A-Za-z0-9-_]+)', '', tweet)      # remove tweeted at
    tweet = re.sub(r'\[user\]', '', tweet )                      # remove [user]
    return tweet

def remove_hashtags(tweet):
    """Takes a string and removes any hash tags"""
    tweet = re.sub('(#[A-Za-z]+[A-Za-z0-9-_]+)', '', tweet)      # remove hash tags
    return tweet

def remove_av(tweet):
    """Takes a string and removes AUDIO/VIDEO tags or labels"""
    tweet = re.sub('VIDEO:', '', tweet)  # remove 'VIDEO:' from start of tweet
    tweet = re.sub('AUDIO:', '', tweet)  # remove 'AUDIO:' from start of tweet
    return tweet

def remove_emojis(tweet):
    emoj = re.compile("["
        u"\U00002700-\U000027BF"  # Dingbats
        u"\U0001F600-\U0001F64F"  # Emoticons
        u"\U00002600-\U000026FF"  # Miscellaneous Symbols
        u"\U0001F300-\U0001F5FF"  # Miscellaneous Symbols And Pictographs
        u"\U0001F900-\U0001F9FF"  # Supplemental Symbols and Pictographs
        u"\U0001FA70-\U0001FAFF"  # Symbols and Pictographs Extended-A
        u"\U00010000-\U0010FFFF"
        u"\U0001F680-\U0001F6FF"  # Transport and Map Symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642"
        u"\u2600-\u2B55"
        u"\ufe0f"  # dingbats

                      "]+", re.UNICODE)
    return re.sub(emoj, '', tweet)

Aplicación de las funciones

In [ ]:
campo_texto = 'text'

train_df[campo_texto] = train_df[campo_texto].str.lower()
valid_df[campo_texto] = valid_df[campo_texto].str.lower()
test_df[campo_texto] = test_df[campo_texto].str.lower()

#train_df[campo_texto] = train_df[campo_texto].apply(remove_links)
#valid_df[campo_texto] = valid_df[campo_texto].apply(remove_links)
#test_df[campo_texto] = test_df[campo_texto].apply(remove_links)

#train_df[campo_texto] = train_df[campo_texto].apply(remove_users)
#valid_df[campo_texto] = valid_df[campo_texto].apply(remove_users)
#test_df[campo_texto] = test_df[campo_texto].apply(remove_users)

#train_df[campo_texto] = train_df[campo_texto].apply(remove_hashtags)
#valid_df[campo_texto] = valid_df[campo_texto].apply(remove_hashtags)
#test_df[campo_texto] = test_df[campo_texto].apply(remove_hashtags)

#train_df[campo_texto] = train_df[campo_texto].apply(remove_emojis)
#valid_df[campo_texto] = valid_df[campo_texto].apply(remove_emojis)
#test_df[campo_texto] = test_df[campo_texto].apply(remove_emojis)

train_df

## Creación de los Modelos

In [ ]:
# Creacion por perspectiva, quedandose con el mayoritario para cada perspectiva
etiqueta = 'labels_task5' ###

# Cuenta la etiqueta más usada
def most_common_label(labels):
    counts = {}
    for label in labels:
        counts[label] = counts.get(label, 0) + 1
    return max(counts, key=counts.get)

def create_perspectivist_datasets(data, perspective=str):
    c = 0
    dataset_dict = {}
    perspectives = list(set(data[perspective].to_list()[1]))

    while c < len(perspectives):
        profile = perspectives[c]

        df = pd.DataFrame()
        ann = 'persp_'+profile
        mv = []
        indices = []

        for i, l in enumerate(data[etiqueta].to_list()):
            labels = []
            for id, label in enumerate(l):
                if data[perspective].to_list()[i][id] == profile:
                    if l[id] == 'DIRECT':
                        labels.append(1)
                    elif l[id] == 'JUDGEMENTAL':
                        labels.append(0)
            if labels != []:
                mas_usado = most_common_label(labels)
                mv.append(mas_usado)
                indices.append(i)

        df['labels'] = mv
        df['text'] = [data['text'].to_list()[idx] for idx in indices] # Obtener los textos relevantes

        # Balanceo de clases
        class_counts = df['labels'].value_counts()
        majority_class = class_counts.idxmax()
        minority_class = class_counts.idxmin()
        majority_count = class_counts[majority_class]
        minority_count = class_counts[minority_class]
        if majority_count > minority_count:
            target_majority_count = int(minority_count * 1)
            majority_indices = df[df['labels'] == majority_class].index
            balanced_majority_indices = np.random.choice(majority_indices, target_majority_count, replace=False)
            balanced_indices = df[df['labels'] == minority_class].index.union(balanced_majority_indices)
            df_balanced = df.loc[balanced_indices]
        else:
            df_balanced = df.copy()

        df_balanced.reset_index(drop=True, inplace=True)
        dataset_dict[ann] = df_balanced

        c += 1

    return dataset_dict


perspectiva = 'gender_annotators'
training_dict_gender = create_perspectivist_datasets(train_df, perspective=perspectiva)

perspectiva = 'age_annotators'
training_dict_age = create_perspectivist_datasets(train_df, perspective=perspectiva)

perspectiva = 'study_levels_annotators'
training_dict_study = create_perspectivist_datasets(train_df, perspective=perspectiva)

perspectiva = 'ethnicities_annotators'
training_dict_ethnicities = create_perspectivist_datasets(train_df, perspective=perspectiva)

In [ ]:
# Comprobar balanceo de clases
training = [training_dict_gender, training_dict_age, training_dict_study, training_dict_ethnicities]

for dataset in training:
    counts = {}
    for persp, df in dataset.items():
        counts[persp] = df['labels'].value_counts()

    print(counts)
    print()

In [ ]:
# Eleccion del modelo
model_checkpoint = "xlm-roberta-base"
#model_checkpoint = 'bert-base-multilingual-uncased'
#model_checkpoint = 'annahaz/xlm-roberta-base-misogyny-sexism-indomain-mix-bal'

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [ ]:
datasets_dict = {
    'persp_M': {'train': training_dict_gender['persp_M']},
    'persp_F': {'train': training_dict_gender['persp_F']},
    'persp_23-45': {'train': training_dict_age['persp_23-45']},
    'persp_18-22': {'train': training_dict_age['persp_18-22']},
    'persp_46+': {'train': training_dict_age['persp_46+']},
    'persp_Bachelor’s degree': {'train': training_dict_study['persp_Bachelor’s degree']},
    'persp_High school degree or equivalent': {'train': training_dict_study['persp_High school degree or equivalent']},
    'persp_White or Caucasian': {'train': training_dict_ethnicities['persp_White or Caucasian']},
}

# Reseteado de indices
for dataset_type in datasets_dict.values():
    for dataset in dataset_type.values():
        dataset.reset_index(drop=True, inplace=True)


datasets_dict

In [ ]:
nombre_etiqueta = 'labels_task5'

def set_labels(records):
  if records[nombre_etiqueta] == 'JUDGEMENTAL':
    label = 0
  else:
    label = 1
  return {'labels': label}

val_df = Dataset.from_pandas(val_df)
val_df.reset_format()
val_df = val_df.map(set_labels)

## Definición de Métricas

In [ ]:
# Función para realizar distintas métricas en ejecución

def compute_metrics(eval_pred):

  ##############
  ## predictions son logits, que son tuplas de la forma [valor1, valor2]
  ## Por ejemplo [-1.5606991,  1.6122842] significa que ha predicho eso para un documento
  ## Eso es lo que pasa a la última capa del transformer (softmax si es binario)
  ## Por eso se utiliza el índice del valor máximo de la tupla, para decir que esa es la clase que predice

  ## label_ids = [0, 1, 1, 0, 1]  # Etiquetas reales
  ## predictions = [
  ##  [0.8, 0.2],  # Predicciones para la primera instancia
  ##  [0.3, 0.7],  # Predicciones para la segunda instancia
  ##  [0.1, 0.9],  # Predicciones para la tercera instancia
  ##  [0.9, 0.1],  # Predicciones para la cuarta instancia
  ##  [0.4, 0.6],  # Predicciones para la quinta instancia
  ##           ]

  ##############

  labels = eval_pred.label_ids
  preds = eval_pred.predictions.argmax(-1)

  # Compute precision, recall, F1-score, and support
  precision, recall, f1, _ = sk.metrics.precision_recall_fscore_support(labels, preds, average="macro")

  # Calculate F1-score for the minority class (label = 1)
  f1_minoritaria= f1_score(labels, preds, pos_label=1)

  # Calculate F1-score for the majority class (label = 0)
  f1_mayoritaria = f1_score(labels, preds, pos_label=0)

  # Calculate accuracy
  acc = sk.metrics.accuracy_score(labels, preds)

  # Calculate Area Under the Curve (AUC)
  AUC = roc_auc_score(labels, preds)

  # Calculate Precision-Recall Area Under the Curve (AUC)
  PREC_REC = average_precision_score(labels, preds)

  return {
      'accuracy': acc,
      'f1': f1,
      'precision': precision,
      'recall': recall,
      'AUC': AUC,
      'f1_minoritaria': f1_minoritaria,
      'f1_mayoritaria': f1_mayoritaria,
      'PREC_REC': PREC_REC
  }

## Entrenamiento del Modelo

In [ ]:
# Se carga el modelo preentrenado
n_labels = 2

# El uso de una función de inicialización facilita la repetición del entrenamiento
# Se puede usar la misma función de inicialización en diferentes ejecuciones del código o en configuraciones de entrenamiento diferentes
# Esto facilita la repetición del entrenamiento y la reproducibilidad, ya que se puede inicializar el modelo
# de la misma manera en cada ejecución.

def model_init():
    return AutoModelForSequenceClassification.from_pretrained(model_checkpoint,
                                                              num_labels = n_labels) #, return_dict = True )
                                                              # use_auth_token = 'token propio de HugginFace')

In [ ]:
# Para saber el nombre del modelo
model_name = model_checkpoint.split("/")[-1]
model_name

Fine-tuning

In [ ]:
# Selección de hiperparámetros
BATCH_SIZE = 32
NUM_TRAIN_EPOCHS = 15
LEARNING_RATE = 3e-5
MAX_LENGTH = 128
WEIGHT_DECAY = 0.1

In [ ]:
def tokenize_data(examples):
  return tokenizer(examples[campo_texto], truncation=True, max_length=MAX_LENGTH, padding=True)

In [ ]:
from transformers import Trainer, TrainingArguments

optim = ["adamw_hf", "adamw_torch", "adamw_apex_fused", "adafactor", "adamw_torch_xla"]
MAX_LENGTH = 128

training_args = TrainingArguments(
    output_dir = 'results',
    num_train_epochs = NUM_TRAIN_EPOCHS,
    learning_rate = LEARNING_RATE,
    per_device_train_batch_size = BATCH_SIZE,
    per_device_eval_batch_size = BATCH_SIZE,
    load_best_model_at_end = True,
    metric_for_best_model = 'recall', # Cambiar la metrica por la que queremos ajustar
    weight_decay = WEIGHT_DECAY,
    evaluation_strategy = 'epoch',
    save_strategy = 'epoch',
    save_total_limit = 3,
    optim = optim[1],
    push_to_hub = False
)

output_dir = '/Dataset/Modelos/Bert_Original'

# Entrenamiento de cada modelo por separado
for task_name, dataset in datasets_dict.items():
    model_output_dir = f"{output_dir}/modelo_{task_name}"

    # Define el conjunto de datos de entrenamiento específico para esta tarea
    train_dataset = dataset['train']
    valid_dataset = val_df

    train_dataset = Dataset.from_pandas(train_dataset)

    # Reseteamos el formato para que no haya fallos
    train_dataset.reset_format()
    valid_dataset.reset_format()

    columns_train = train_dataset.column_names  # Coge todas las columnas
    columns_valid = valid_dataset.column_names  # Coge todas las columnas
    columns_train.remove("labels") # Elimina la columna "labels"
    columns_valid.remove("labels") # Elimina la columna "labels"

    # Hace la tokenización y elimina todas las columnas que no se necesitan
    encoded_train_dataset = train_dataset.map(tokenize_data, batched=True, remove_columns=columns_train)
    encoded_valid_dataset = valid_dataset.map(tokenize_data, batched=True, remove_columns=columns_valid)

    # Inicializa el entrenador
    trainer = Trainer(
        model_init = model_init,
        args=training_args,
        compute_metrics = compute_metrics,
        train_dataset=encoded_train_dataset,
        eval_dataset=encoded_valid_dataset,
        callbacks=[EarlyStoppingCallback(early_stopping_patience=3)],
        tokenizer=tokenizer,
    )

    # Entrena el modelo
    trainer.train()

    # Guarda el modelo entrenado
    trainer.save_model(model_output_dir)

## Evaluación del Modelo

### Lectura de Test

In [ ]:
# Cargamos los datos de entrenamiento test
test_data_path = '/content/drive/MyDrive/Dataset/Task 5/test_task5_hard.json'

# Los transformamos en Dataframes
test_df = pd.read_json(train_data_path, orient='index')
test_df

### Preprocesado Test

In [ ]:
# Funciones de limpieza
import re

def remove_links(tweet):
    """Takes a string and removes web links from it"""
    tweet = re.sub(r'http\S+', '', tweet)        # remove http links
    tweet = re.sub(r'bit.ly/\S+', '', tweet)     # remove bitly links
    tweet = re.sub(r'\[link\]', '', tweet )      # remove [link]
    tweet = re.sub(r'\[url\]', '', tweet )       # remove [url]
    tweet = re.sub(r'pic.twitter\S+','', tweet)
    return tweet

def remove_users(tweet):
    """Takes a string and removes retweet and @user information"""
    tweet = re.sub('(RT\s@[A-Za-z]+[A-Za-z0-9-_]+)', '', tweet)  # remove re-tweet
    tweet = re.sub('(@[A-Za-z]+[A-Za-z0-9-_]+)', '', tweet)      # remove tweeted at
    tweet = re.sub(r'\[user\]', '', tweet )                      # remove [user]
    return tweet

def remove_hashtags(tweet):
    """Takes a string and removes any hash tags"""
    tweet = re.sub('(#[A-Za-z]+[A-Za-z0-9-_]+)', '', tweet)      # remove hash tags
    return tweet

def remove_av(tweet):
    """Takes a string and removes AUDIO/VIDEO tags or labels"""
    tweet = re.sub('VIDEO:', '', tweet)  # remove 'VIDEO:' from start of tweet
    tweet = re.sub('AUDIO:', '', tweet)  # remove 'AUDIO:' from start of tweet
    return tweet

def remove_emojis(tweet):
    emoj = re.compile("["
        u"\U00002700-\U000027BF"  # Dingbats
        u"\U0001F600-\U0001F64F"  # Emoticons
        u"\U00002600-\U000026FF"  # Miscellaneous Symbols
        u"\U0001F300-\U0001F5FF"  # Miscellaneous Symbols And Pictographs
        u"\U0001F900-\U0001F9FF"  # Supplemental Symbols and Pictographs
        u"\U0001FA70-\U0001FAFF"  # Symbols and Pictographs Extended-A
        u"\U00010000-\U0010FFFF"
        u"\U0001F680-\U0001F6FF"  # Transport and Map Symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642"
        u"\u2600-\u2B55"
        u"\ufe0f"  # dingbats

                      "]+", re.UNICODE)
    return re.sub(emoj, '', tweet)

In [ ]:
campo_texto = 'text'

test_df[campo_texto] = test_df[campo_texto].str.lower()
#test_df[campo_texto] = test_df[campo_texto].apply(remove_links)
#test_df[campo_texto] = test_df[campo_texto].apply(remove_users)
#test_df[campo_texto] = test_df[campo_texto].apply(remove_hashtags)
#test_df[campo_texto] = test_df[campo_texto].apply(remove_emojis)

In [ ]:
# Lo pasamos a objeto dataset
test_dataset = Dataset.from_pandas(test_df)
test_dataset

In [ ]:
nombre_etiqueta = 'labels_task5'

def set_labels(records):
  if records[nombre_etiqueta] == 'JUDGEMENTAL':
    label = 0
  else:
    label = 1
  return {'labels': label}

# Pasamos la etiqueta a label y le damos formato numérico
test_dataset = test_dataset.map(set_labels)  # La función set_labels ya se definió en el entrenamiento
test_dataset

### Cargar Modelos

In [ ]:
# Se carga los modelos que se han entrenado
modelos = ["persp_M", "persp_F", "persp_23-45", "persp_18-22", "persp_46+", "persp_Bachelor’s degree", "persp_High school degree or equivalent", "persp_White or Caucasian"]

model_path = '/content/drive/MyDrive/Dataset/Modelos/Task 5/Modelo1/modelo_'

loaded_models = {}
for modelo in modelos:
    model = AutoModelForSequenceClassification.from_pretrained(model_path + modelo)
    loaded_models[modelo] = model

In [ ]:
loaded_models

### Realizar Predicciones

In [ ]:
# Añadimos la función de evaluación
def compute_metrics(pred):

  labels = pred[0]
  preds = pred[1]
  precision, recall, f1, _ = sk.metrics.precision_recall_fscore_support(labels, preds, average="macro")
  acc = sk.metrics.accuracy_score(labels, preds)
  AUC = roc_auc_score(labels, preds)
  PREC_REC = average_precision_score(labels, preds)
  return { 'accuracy': acc, 'f1': f1, 'precision': precision,
          'recall': recall, 'AUC': AUC, 'PREC_REC': PREC_REC }

In [ ]:
# Eleccion del modelo
#model_checkpoint = "xlm-roberta-base"
#tokenizer1 = AutoTokenizer.from_pretrained(model_checkpoint)
model_checkpoint = 'bert-base-multilingual-uncased'
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

#tokenizer = [tokenizer1, tokenizer2, tokenizer2, tokenizer1, tokenizer1, tokenizer2, tokenizer1, tokenizer1] # Si es para modelo Bert/Roberta
#i = 0

# Crear un diccionario de pipelines para cada modelo
pipes = {}
for modelo_nombre, modelo in loaded_models.items():
    #pipe = pipeline("text-classification", model=modelo, tokenizer=tokenizer[i], device=0)
    #i += 1
    pipe = pipeline("text-classification", model=modelo, tokenizer=tokenizer, device=0)
    pipes[modelo_nombre] = pipe

# Definir la función para hacer predicciones para todos los modelos a la vez
def get_predictions_for_all_models(records):
    predictions_all_models = {}
    for modelo_nombre, pipe in pipes.items():
        result = pipe(records[campo_texto], truncation=True)
        pred_label = result[0]['label']
        score_label = result[0]['score']

        if pred_label == 'LABEL_0':
            pred_label = 0
        else:
            pred_label = 1

        predictions_all_models[modelo_nombre] = pred_label

    return predictions_all_models

# Aplicar la función a los conjuntos de datos de prueba y validación
test_dataset_predicted_all_models = test_dataset.map(get_predictions_for_all_models)

# Imprimir el primer ejemplo de los conjuntos de datos con predicciones para todos los modelos
print("Primer ejemplo del conjunto de datos de prueba con predicciones para todos los modelos:")
print(test_dataset_predicted_all_models[0])

In [ ]:
test_dataset_predicted_all_models

In [ ]:
test_dataset_predicted_all_models.set_format('pandas')
df_test = test_dataset_predicted_all_models[:]
df_test

### Evaluacion de Predicciones Individuales

In [ ]:
test_labels = df_test['labels'].values.tolist()

# Iterar sobre cada modelo para evaluarlo
for modelo_nombre, pipe in pipes.items():
    # Hacer predicciones utilizando el modelo en el conjunto de datos de prueba
    predicciones = df_test[modelo_nombre].values.tolist()
    eval_pred_test = [test_labels, predicciones]

    # Calcular las métricas utilizando la función compute_metrics
    metrics = compute_metrics(eval_pred_test)

    # Imprimir las métricas para el modelo actual
    print(f"Métricas para el modelo '{modelo_nombre}':")
    #print("Accuracy:", metrics['accuracy'])
    print("F1 score:", metrics['f1'])
    #print("Precision:", metrics['precision'])
    #print("Recall:", metrics['recall'])
    #print("AUC:", metrics['AUC'])
    #print("Precision-Recall AUC:", metrics['PREC_REC'])
    print("\n")

### Prediccion Conjunta

In [ ]:
pesos = {
    "persp_M": {"Peso": 1},
    "persp_F": {"Peso": 1},
    "persp_23-45": {"Peso": 1},
    "persp_18-22": {"Peso": 1},
    "persp_46+": {"Peso": 1},
    "persp_Bachelor’s degree": {"Peso": 1},
    "persp_High school degree or equivalent": {"Peso": 1},
    "persp_White or Caucasian": {"Peso": 1},
}

# Calcula la predicción conjunta
def calcular_prediccion_conjunta(df_test, pesos):
    suma_ponderada = 0
    for modelo in modelos:
        peso = pesos[modelo]["Peso"]
        pred = df_test[modelo]
        suma_ponderada += pred * peso

    # Divide la suma ponderada por el número de modelos
    pred_conjunta = suma_ponderada / len(modelos)

    # Aplica el umbral de 0.5 a cada elemento de pred_conjunta
    for i in range(len(pred_conjunta)):
        if pred_conjunta[i] > 0.5:
            pred_conjunta[i] = 1
        else:
            pred_conjunta[i] = 0

    return pred_conjunta

# Calculo de predicciones conjuntas
predicciones_finales = calcular_prediccion_conjunta(df_test, pesos)

# Imprimir las predicciones finales
print("Predicciones finales:")
predicciones_finales

In [ ]:
test_labels = df_test['labels'].values.tolist()
eval_pred_test = [test_labels, predicciones_finales]

# Calcular las métricas utilizando la función compute_metrics
metrics = compute_metrics(eval_pred_test)

# Imprimir las métricas para el modelo actual
print(f"Métricas para el modelo '{modelo_nombre}':")
print("Accuracy:", metrics['accuracy'])
print("F1 score:", metrics['f1'])
print("Precision:", metrics['precision'])
print("Recall:", metrics['recall'])
print("AUC:", metrics['AUC'])
print("Prec Rec:", metrics['PREC_REC'])
print("\n")

## Calculo de Balance de Pesos

In [ ]:
import itertools

# Función para calcular las métricas dado un conjunto de pesos
def calcular_metricas_con_pesos(df_test, pesos):
    predicciones_finales = calcular_prediccion_conjunta(df_test, pesos)
    test_labels = df_test['labels'].values.tolist()
    eval_pred_test = [test_labels, predicciones_finales]
    metrics = compute_metrics(eval_pred_test)
    return metrics

# Obtener la lista de modelos del diccionario de pesos
modelos = list(pesos.keys())

# Obtener todas las combinaciones posibles de pesos que suman 8
combinaciones_pesos = []
pesos_posibles = [0.5, 0.75, 1, 1.25, 1.5, 1.75]

for p in itertools.product(pesos_posibles, repeat=len(modelos)):
    if sum(p) == 8:
        combinaciones_pesos.append(p)

# Inicializar las mejores métricas y pesos
mejor_metricas = None
mejor_pesos = None
it = 0

# Iterar sobre todas las combinaciones de pesos
for pesos_combinacion in combinaciones_pesos:
    pesos = {modelo: {"Peso": peso} for modelo, peso in zip(modelos, pesos_combinacion)}
    metricas = calcular_metricas_con_pesos(df_test, pesos)
    print(it)
    it += 1

    # Actualizar las mejores métricas y pesos si las métricas actuales son mejores
    if mejor_metricas is None or metricas['f1'] > mejor_metricas['f1']:
        mejor_metricas = metricas
        mejor_pesos = pesos

# Imprimir las métricas del mejor modelo
print("Mejor conjunto de pesos:")
print(mejor_pesos)
print("Métricas del mejor modelo:")
print("Accuracy:", mejor_metricas['accuracy'])
print("F1 score:", mejor_metricas['f1'])
print("Precision:", mejor_metricas['precision'])
print("Recall:", mejor_metricas['recall'])
print("AUC:", mejor_metricas['AUC'])
print("Prec Rec:", mejor_metricas['PREC_REC'])